In [ ]:
import os
import cv2
import numpy as np
from glob import glob
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
)
import pandas as pd
from typing import List, Tuple

# -----------------------------
# CONFIGURATION
# -----------------------------
DATA_PATH = r"prnu-camera-source-detection/data"
TARGET_SIZE = (1024, 768)
THRESHOLD = 2
N_SHARES = 5
PRIME = 2**61 - 1
SCALE = 1500

# -----------------------------
# PRNU Extraction
# -----------------------------
def extract_prnu(image_path: str, target_size: Tuple[int, int] = TARGET_SIZE) -> np.ndarray:
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"⚠️ Cannot read: {image_path}")
        return None
    img = cv2.resize(img, target_size)
    denoised = cv2.GaussianBlur(img, (3, 3), 0)
    prnu = (img.astype(np.float32) - denoised.astype(np.float32))
    prnu -= np.mean(prnu)
    prnu /= (np.std(prnu) + 1e-8)
    return prnu


def compute_fingerprint(image_paths: List[str], target_size: Tuple[int, int] = TARGET_SIZE) -> np.ndarray:
    prnus = [extract_prnu(p, target_size) for p in image_paths if extract_prnu(p) is not None]
    if not prnus:
        raise ValueError("No valid PRNU images found.")
    return np.mean(prnus, axis=0)

# -----------------------------
# Shamir Secret Sharing
# -----------------------------
def _eval_poly(coeffs: List[int], x: int, prime: int) -> int:
    res, power = 0, 1
    for c in coeffs:
        res = (res + c * power) % prime
        power = (power * x) % prime
    return res

def shamir_split(secret_int: int, t: int, n: int, prime: int = PRIME):
    coeffs = [secret_int] + [int(np.random.randint(0, prime)) for _ in range(t - 1)]
    shares = [(i, _eval_poly(coeffs, i, prime)) for i in range(1, n + 1)]
    return shares

def _lagrange_interpolate(points: List[Tuple[int, int]], prime: int = PRIME) -> int:
    total = 0
    for i, (xi, yi) in enumerate(points):
        num, den = 1, 1
        for j, (xj, _) in enumerate(points):
            if i != j:
                num = (num * (-xj)) % prime
                den = (den * (xi - xj)) % prime
        inv_den = pow(den, -1, prime)
        total = (total + yi * num * inv_den) % prime
    return total

def shamir_reconstruct(points: List[Tuple[int, int]], prime: int = PRIME) -> int:
    return _lagrange_interpolate(points, prime)

# -----------------------------
# Vector Encryption / Decryption
# -----------------------------
def encrypt_vector(vec: np.ndarray, threshold: int, shares: int) -> List[List[int]]:
    scaled = np.round(vec * SCALE).astype(np.int64).flatten()
    servers = [[] for _ in range(shares)]
    for val in scaled:
        val_mod = int(val % PRIME)
        parts = shamir_split(val_mod, threshold, shares, PRIME)
        for s_idx, (_, y) in enumerate(parts):
            servers[s_idx].append(y)
    return servers


def reconstruct_vector(servers: List[List[int]], threshold: int, shape: Tuple[int, int]) -> np.ndarray:
    chosen = [(i + 1, servers[i]) for i in range(threshold)]
    reconstructed = []
    for idx in range(len(servers[0])):
        points = [(x, vals[idx]) for (x, vals) in chosen]
        val = shamir_reconstruct(points, PRIME)
        reconstructed.append(val)
    arr = np.array(reconstructed, dtype=np.float64).reshape(shape)
    return arr / SCALE

# -----------------------------
# Correlation
# -----------------------------
def corr2d(x: np.ndarray, y: np.ndarray) -> float:
    x = x - np.mean(x)
    y = y - np.mean(y)
    numerator = np.sum(x * y)
    denominator = np.sqrt(np.sum(x**2) * np.sum(y**2))
    if denominator == 0:
        return 0.0
    return float(numerator / denominator)

# -----------------------------
# Pipeline
# -----------------------------
def process_phone(phone_path: str) -> Tuple[np.ndarray, List[str]]:
    fp_imgs = glob(os.path.join(phone_path, "images/fingerprint_set", "*.[jJ][pP][gG]")) + \
               glob(os.path.join(phone_path, "images/fingerprint_set", "*.[jJ][pP][eE][gG]"))

    q_imgs = glob(os.path.join(phone_path, "images/query_set", "*.[jJ][pP][gG]")) + \
              glob(os.path.join(phone_path, "images/query_set", "*.[jJ][pP][eE][gG]"))
    if not fp_imgs:
        raise ValueError(f"No fingerprint images for {phone_path}")
    fingerprint = compute_fingerprint(fp_imgs)
    return fingerprint, q_imgs


def run_sss_prnu_pipeline(data_path: str):
    phones = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]
    fingerprints, servers_store, queries = {}, {}, {}

    print("🔹 Extracting fingerprints & encrypting...\n")
    for phone in phones:
        try:
            fp, qset = process_phone(os.path.join(data_path, phone))
            fingerprints[phone] = fp
            queries[phone] = qset
            servers_store[phone] = encrypt_vector(fp, THRESHOLD, N_SHARES)
        except Exception as e:
            print(f"⚠️ Skipping {phone}: {e}")

    print("\n--- Matching Report ---")
    results = []
    for true_phone, qset in queries.items():
        for q_img in qset:
            q_prnu = extract_prnu(q_img)
            if q_prnu is None:
                continue
            q_servers = encrypt_vector(q_prnu, THRESHOLD, N_SHARES)
            q_recon = reconstruct_vector(q_servers, THRESHOLD, q_prnu.shape)

            best_score, best_match = -1, None
            for phone, fp_servers in servers_store.items():
                fp_recon = reconstruct_vector(fp_servers, THRESHOLD, fingerprints[phone].shape)
                score = corr2d(fp_recon, q_recon)
                if score > best_score:
                    best_score = score
                    best_match = phone

            results.append((os.path.basename(q_img), true_phone, best_match, best_score))
            print(f"{os.path.basename(q_img)} | True: {true_phone:<15} | Predicted: {best_match:<15} | Corr: {best_score:.4f}")

    # -----------------------------
    # Accuracy and Metrics Report
    # -----------------------------
    print("\n--- Detailed Accuracy Report ---")
    df = pd.DataFrame(results, columns=["Image", "True", "Predicted", "Correlation"])
    all_true = df["True"].tolist()
    all_pred = df["Predicted"].tolist()

    overall_acc = accuracy_score(all_true, all_pred)
    overall_prec = precision_score(all_true, all_pred, average="macro", zero_division=0)
    overall_rec = recall_score(all_true, all_pred, average="macro", zero_division=0)
    overall_f1 = f1_score(all_true, all_pred, average="macro", zero_division=0)

    print(f"\n✅ Overall Accuracy : {overall_acc*100:.2f}%")
    print(f"✅ Precision        : {overall_prec*100:.2f}%")
    print(f"✅ Recall           : {overall_rec*100:.2f}%")
    print(f"✅ F1-Score         : {overall_f1*100:.2f}%\n")

    print("--- Per-Phone Metrics ---")
    for phone in phones:
        preds = df[df["True"] == phone]["Predicted"]
        trues = df[df["True"] == phone]["True"]
        if len(trues) == 0:
            continue
        acc = np.mean(preds == trues)
        print(f"{phone:<25}: {acc*100:.2f}% ({len(trues)} queries)")

    print("\n--- Confusion Matrix ---")
    cm = confusion_matrix(all_true, all_pred, labels=phones)
    cm_df = pd.DataFrame(cm, index=phones, columns=phones)
    print(cm_df)

# -----------------------------
# Run
# -----------------------------
if __name__ == "__main__":
    run_sss_prnu_pipeline(DATA_PATH)